In [18]:
import json
import os
from typing import Annotated, List, Dict, Any
from datetime import datetime
import uuid

from IPython.display import display, HTML, Markdown
from dotenv import load_dotenv

# Azure AI Search
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SearchField,
    VectorSearchAlgorithmMetric
)

# Mem0
from mem0 import Memory

# Semantic Kernel
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.functions import kernel_function
from semantic_kernel.contents import ChatHistory
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread

In [19]:
# Load environment variables
load_dotenv()

# Azure OpenAI Configuration
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")  # Use a recent API version


# Azure AI Search Configuration
search_service_endpoint = "https://aisearchexporationpremium.search.windows.net"
search_api_key = "4R8Vb8fOjkxF3zlDvCfgKWtgyH3mO1ubruhV0oUxcRAzSeAakMGb"


# Index names
travel_index_name = "travel-hotels"
memory_index_name = "mem0-memories"

In [20]:
# Initialize search clients
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=AzureKeyCredential(search_api_key)
)

# Create travel data index if it doesn't exist
travel_fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="name", type=SearchFieldDataType.String),
    SearchableField(name="description", type=SearchFieldDataType.String),
    SearchableField(name="location", type=SearchFieldDataType.String),
    SearchableField(name="amenities", type=SearchFieldDataType.String),
    SimpleField(name="price_per_night", type=SearchFieldDataType.Double),
    SimpleField(name="rating", type=SearchFieldDataType.Double),
    SearchableField(name="tags", type=SearchFieldDataType.String, collection=True)
]

travel_index = SearchIndex(name=travel_index_name, fields=travel_fields)

try:
    index_client.get_index(travel_index_name)
    print(f"✅ Index '{travel_index_name}' already exists")
except:
    index_client.create_index(travel_index)
    print(f"✅ Created index '{travel_index_name}'")

# Initialize search client for travel data
travel_search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=travel_index_name,
    credential=AzureKeyCredential(search_api_key)
)

✅ Index 'travel-hotels' already exists


In [21]:
# Add sample travel data
sample_hotels = [
    {
        "id": "1",
        "name": "Le Meurice Paris",
        "description": "Luxury palace hotel with Michelin-starred dining and views of the Tuileries Garden",
        "location": "Paris, France",
        "amenities": "Spa, Michelin Restaurant, Concierge, Room Service, Fitness Center",
        "price_per_night": 850,
        "rating": 4.8,
        "tags": ["luxury", "romantic", "historic", "fine-dining", "spa"]
    },
    {
        "id": "2",
        "name": "Four Seasons Maui",
        "description": "Beachfront resort with world-class spa and family-friendly activities",
        "location": "Maui, Hawaii",
        "amenities": "Beach Access, Kids Club, Multiple Pools, Spa, Golf Course",
        "price_per_night": 695,
        "rating": 4.7,
        "tags": ["beach", "family-friendly", "resort", "spa", "golf"]
    },
    {
        "id": "3",
        "name": "Aman Tokyo",
        "description": "Minimalist luxury hotel with panoramic city views and traditional onsen",
        "location": "Tokyo, Japan",
        "amenities": "Onsen, City Views, Fine Dining, Spa, Business Center",
        "price_per_night": 780,
        "rating": 4.9,
        "tags": ["luxury", "business", "spa", "city", "minimalist"]
    },
    {
        "id": "4",
        "name": "Hotel Sacher Vienna",
        "description": "Historic hotel home of the original Sachertorte with elegant rooms",
        "location": "Vienna, Austria",
        "amenities": "Historic Cafe, Concierge, Accessible Rooms, Pet-Friendly",
        "price_per_night": 420,
        "rating": 4.6,
        "tags": ["historic", "accessible", "pet-friendly", "cultural", "cafe"]
    },
    {
        "id": "5",
        "name": "Fairmont Whistler",
        "description": "Ski-in/ski-out resort with family suites and mountain views",
        "location": "Whistler, Canada",
        "amenities": "Ski Access, Family Suites, Heated Pool, Kids Programs",
        "price_per_night": 380,
        "rating": 4.5,
        "tags": ["ski", "family-friendly", "mountain", "resort", "accessible"]
    }
]

# Upload hotels to search index
travel_search_client.upload_documents(documents=sample_hotels)
print(f"✅ Uploaded {len(sample_hotels)} hotels to search index")

✅ Uploaded 5 hotels to search index


In [22]:
mem0_config = {
    "llm": {
        "provider": "azure_openai",
        "config": {
            "model": azure_openai_deployment,
            "temperature": 0.2,
            "max_tokens": 1500,
            "azure_kwargs": {
                "azure_deployment": azure_openai_deployment,
                "api_version": api_version,
                "azure_endpoint": azure_openai_endpoint,
                "api_key": azure_openai_api_key,
            }
        }
    },
    "vector_store": {
        "provider": "azure_ai_search",
        "config": {
            "service_name": search_service_endpoint.split("//")[1].split(".")[0],
            "api_key": search_api_key,
            "collection_name": "mem0",
            "embedding_model_dims": 1536
        }
    },
    "embedder": {
        "provider": "azure_openai",
        "config": {
            "model": "text-embedding-ada-002",  # Your embedding deployment name
            "azure_kwargs": {
                "azure_deployment": "text-embedding-ada-002",  # Update if different
                "api_version": api_version,
                "azure_endpoint": azure_openai_endpoint,
                "api_key": azure_openai_api_key,
            }
        }
    }
}

# Initialize Mem0
memory = Memory.from_config(mem0_config)
# Test the memory system
print("🧪 Testing Mem0 setup...")
test_messages = [
    {"role": "user", "content": "I prefer luxury hotels with spa services."},
    {"role": "assistant", "content": "I'll remember you prefer luxury hotels with spa services for future recommendations."}
]
memory.add(test_messages, user_id="test_user",
           metadata={"category": "preferences"})
test_memories = memory.get_all(user_id="test_user")
print(f"✅ Mem0 test successful! Found {len(test_memories)} memories")

🧪 Testing Mem0 setup...
✅ Mem0 test successful! Found 1 memories


In [23]:
test_memories = memory.get_all(user_id="test_user")
print(test_memories)

{'results': [{'id': '070ee423-ae73-41a7-ac4d-baa780fd4cb4', 'memory': 'Prefers luxury hotels with spa services', 'hash': 'b7af1e635295620d37e156b22d228fe8', 'metadata': {'category': 'preferences'}, 'created_at': '2025-09-24T10:26:40.053464-07:00', 'updated_at': None, 'user_id': 'test_user'}]}


In [ ]:
Results from Mem0 directly:
{
    'results': [
    {'id': '070ee423-ae73-41a7-ac4d-baa780fd4cb4', 
    'memory': 'Prefers luxury hotels with spa services', 
    'hash': 'b7af1e635295620d37e156b22d228fe8', 
    'metadata': {'category': 'preferences'}, 
    'created_at': '2025-09-24T10:26:40.053464-07:00', 
    'updated_at': None, 
    'user_id': 'test_user'}
    ]
}

Resutlts from Azure AI Search directly:
{
  "@odata.context": "https://aisearchexporationpremium.search.windows.net/indexes('mem0')/$metadata#docs(*)",
  "@odata.count": 1,
  "value": [
    {
      "@search.score": 1,
      "id": "070ee423-ae73-41a7-ac4d-baa780fd4cb4",
      "user_id": "test_user",
      "run_id": null,
      "agent_id": null,
      "payload": "{\"category\": \"preferences\", 
      \"user_id\": \"test_user\", 
      \"data\": \"Prefers luxury hotels with spa services\", 
      \"hash\": \"b7af1e635295620d37e156b22d228fe8\", 
      \"created_at\": \"2025-09-24T10:26:40.053464-07:00\"
    }"
    }
  ]
}

In [29]:
travel_search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=travel_index_name,
    credential=AzureKeyCredential(search_api_key)
)

search_client = travel_search_client
max_results = 3
query = "luxury hotels with spa services"

results = search_client.search(
    search_text=query,
    top=max_results,
    include_total_count=True
)




hotels = []
for result in results:
    hotels.append({
        "name": result["name"],
        "location": result["location"],
        "description": result["description"],
        "price_per_night": result["price_per_night"],
        "rating": result["rating"],
        "amenities": result["amenities"],
        "tags": result["tags"]
})

print(hotels)

[{'name': 'Four Seasons Maui', 'location': 'Maui, Hawaii', 'description': 'Beachfront resort with world-class spa and family-friendly activities', 'price_per_night': 695.0, 'rating': 4.7, 'amenities': 'Beach Access, Kids Club, Multiple Pools, Spa, Golf Course', 'tags': ['beach', 'family-friendly', 'resort', 'spa', 'golf']}, {'name': 'Aman Tokyo', 'location': 'Tokyo, Japan', 'description': 'Minimalist luxury hotel with panoramic city views and traditional onsen', 'price_per_night': 780.0, 'rating': 4.9, 'amenities': 'Onsen, City Views, Fine Dining, Spa, Business Center', 'tags': ['luxury', 'business', 'spa', 'city', 'minimalist']}, {'name': 'Le Meurice Paris', 'location': 'Paris, France', 'description': 'Luxury palace hotel with Michelin-starred dining and views of the Tuileries Garden', 'price_per_night': 850.0, 'rating': 4.8, 'amenities': 'Spa, Michelin Restaurant, Concierge, Room Service, Fitness Center', 'tags': ['luxury', 'romantic', 'historic', 'fine-dining', 'spa']}]


In [ ]:
[
    {
        'name': 'Four Seasons Maui', 
        'location': 'Maui, Hawaii', 
        'description': 'Beachfront resort with world-class spa and family-friendly activities', 
        'price_per_night': 695.0, 
        'rating': 4.7, 'amenities': 
        'Beach Access, Kids Club, Multiple Pools, Spa, Golf Course', 
        'tags': ['beach', 'family-friendly', 'resort', 'spa', 'golf']
    }, 
    {
        'name': 'Aman Tokyo', 
        'location': 'Tokyo, Japan', 
        'description': 'Minimalist luxury hotel with panoramic city views and traditional onsen', 
        'price_per_night': 780.0, 
        'rating': 4.9, 
        'amenities': 
        'Onsen, City Views, Fine Dining, Spa, Business Center', 
        'tags': ['luxury', 'business', 'spa', 'city', 'minimalist']
    }, 
    {
        'name': 'Le Meurice Paris', 
        'location': 'Paris, France', 
        'description': 'Luxury palace hotel with Michelin-starred dining and views of the Tuileries Garden', 
        'price_per_night': 850.0, 
        'rating': 4.8, 
        'amenities': 'Spa, Michelin Restaurant, Concierge, Room Service, Fitness Center', 
        'tags': ['luxury', 'romantic', 'historic', 'fine-dining', 'spa']
    }
]

In [ ]:
@kernel_function(
        description="Search user's memories for relevant information"
    )
    def search_memories(
        self,
        user_id: Annotated[str, "User identifier"],
        query: Annotated[str,
                         "What to search for (e.g., 'family vacation', 'dietary restrictions')"]
    ) -> Annotated[str, "Relevant memories"]:
        """Search user memories using Mem0"""
        print(f"DEBUG: Searching memories for {user_id} with query: '{query}'")

        try:
            # Let Mem0 handle the search and ranking
            results = self.memory.search(query, user_id=user_id)

            # Handle the dict response with 'results' key
            if isinstance(results, dict) and 'results' in results:
                results = results.get('results', [])

            if not results:
                return f"No memories found for query: {query}"

            # Format results
            memories = []
            for result in results:
                if isinstance(result, dict):
                    memory_text = result.get('memory', str(result))
                    # Include relevance score if available
                    score = result.get('score', None)
                    if score:
                        memories.append(
                            f"{memory_text} (relevance: {score:.2f})")
                    else:
                        memories.append(memory_text)
                else:
                    memories.append(str(result))

            return "Relevant memories:\n- " + "\n- ".join(memories)

        except Exception as e:
            print(f"ERROR: {str(e)}")
            return "No memories found."

In [58]:
#seach index by a query
travel_search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=travel_index_name,
    credential=AzureKeyCredential(search_api_key)
)

search_client = travel_search_client
max_results = 3
query = "luxury hotels with spa services"

results = search_client.search(
    search_text=query,
    top=max_results,
    include_total_count=True
)


hotels = []
for result in results:
    print(result)
    hotels.append({
        "name": result["name"],
        "location": result["location"],
        "description": result["description"],
        "price_per_night": result["price_per_night"],
        "rating": result["rating"],
        "amenities": result["amenities"],
        "tags": result["tags"]
})

print(hotels)

{'rating': 4.7, 'tags': ['beach', 'family-friendly', 'resort', 'spa', 'golf'], 'location': 'Maui, Hawaii', 'name': 'Four Seasons Maui', 'id': '2', 'price_per_night': 695.0, 'amenities': 'Beach Access, Kids Club, Multiple Pools, Spa, Golf Course', 'description': 'Beachfront resort with world-class spa and family-friendly activities', '@search.score': 2.9102762, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}
{'rating': 4.9, 'tags': ['luxury', 'business', 'spa', 'city', 'minimalist'], 'location': 'Tokyo, Japan', 'name': 'Aman Tokyo', 'id': '3', 'price_per_night': 780.0, 'amenities': 'Onsen, City Views, Fine Dining, Spa, Business Center', 'description': 'Minimalist luxury hotel with panoramic city views and traditional onsen', '@search.score': 1.3928729, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}
{'rating': 4.8, 'tags': ['luxury', 'romantic', 'historic', 'fine-dining', 'spa'], 'location': 'Paris, France', 'na

In [ ]:
#Update memory with a new preference

user_id = "amartays"
preference = "I like sea facing hotels."
memory.add(preference, user_id=user_id)

{'results': [{'id': '368f9cb4-502c-4ea0-8bbf-fb29e4a7c886',
   'memory': 'Likes sea-facing hotels',
   'event': 'ADD'}]}

In [ ]:
#retriving data from memory by user_id
user_id = "amartays"
results = memory.get_all(user_id=user_id)
print(results)

#Extracting relevant preferences from memory
user_id = "amartays"

results = memory.get_all(user_id=user_id)
if isinstance(results, dict) and 'results' in results:
    results = results.get('results', [])

memories = []
for result in results:
    if isinstance(result, dict):
        memory_text = result.get('memory', str(result))
        memories.append(memory_text)
    else:
        memories.append(str(result))

print(memories)

{'results': [{'id': '558ee12c-2437-4e76-8dae-0a1cb63c0742', 'memory': 'Likes luxury hotels with spa services', 'hash': 'e4c2a6eb0311bbbed2ea174d6ccf00f5', 'metadata': None, 'created_at': '2025-09-25T22:34:36.022239-07:00', 'updated_at': None, 'user_id': 'amartays'}, {'id': '368f9cb4-502c-4ea0-8bbf-fb29e4a7c886', 'memory': 'Likes sea-facing hotels', 'hash': 'a9d18364bd807d4f9bcbe21156989f8e', 'metadata': None, 'created_at': '2025-09-25T22:35:44.813170-07:00', 'updated_at': None, 'user_id': 'amartays'}]}
['Likes luxury hotels with spa services', 'Likes sea-facing hotels']


In [ ]:
#seach index by a query using mem0
user_id = "amartays"
query = "sea facing hotels"
results = memory.search(query, user_id=user_id)


if isinstance(results, dict) and 'results' in results:
    results = results.get('results', [])

memories = []
for result in results:
    print(result)
    if isinstance(result, dict):
        memory_text = result.get('memory', str(result))
        # Include relevance score if available
        score = result.get('score', None)
        if score:
            memories.append(
            f"{memory_text} (relevance: {score:.2f})")
        else:
            memories.append(memory_text)
    else:
        memories.append(str(result))

print(memories)

{'id': '368f9cb4-502c-4ea0-8bbf-fb29e4a7c886', 'memory': 'Likes sea-facing hotels', 'hash': 'a9d18364bd807d4f9bcbe21156989f8e', 'metadata': None, 'score': 0.9207112, 'created_at': '2025-09-25T22:35:44.813170-07:00', 'updated_at': None, 'user_id': 'amartays'}
{'id': '558ee12c-2437-4e76-8dae-0a1cb63c0742', 'memory': 'Likes luxury hotels with spa services', 'hash': 'e4c2a6eb0311bbbed2ea174d6ccf00f5', 'metadata': None, 'score': 0.85124236, 'created_at': '2025-09-25T22:34:36.022239-07:00', 'updated_at': None, 'user_id': 'amartays'}
['Likes sea-facing hotels (relevance: 0.92)', 'Likes luxury hotels with spa services (relevance: 0.85)']


In [59]:

from azure.search.documents import SearchClient
print("\n\n🔍 VERIFYING MEM0 STORAGE\n")

# Check Azure AI Search directly
mem0_search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name="mem0",
    credential=AzureKeyCredential(search_api_key)
)

try:
    # Count documents in the index
    results = mem0_search_client.search(
        search_text="*", include_total_count=True)
    total_docs = results.get_count()
    print(f"📊 Total documents in Mem0 index: {total_docs}")

    # Show first few documents
    print("\nSample documents:")
    for i, doc in enumerate(results):
        if i < 3:  # Show first 3
            print(f"\nDocument {i+1}:")
            print(f"  ID: {doc.get('id', 'N/A')}")
            print(f"  User ID: {doc.get('user_id', 'N/A')}")
            print(f"  Memory: {doc.get('payload', 'N/A')}")
except Exception as e:
    print(f"❌ Error checking Mem0 index: {str(e)}")
    print("The index might not be created yet or might be empty.")



🔍 VERIFYING MEM0 STORAGE

📊 Total documents in Mem0 index: 3

Sample documents:

Document 1:
  ID: 070ee423-ae73-41a7-ac4d-baa780fd4cb4
  User ID: test_user
  Memory: {"category": "preferences", "user_id": "test_user", "data": "Prefers luxury hotels with spa services", "hash": "b7af1e635295620d37e156b22d228fe8", "created_at": "2025-09-24T10:26:40.053464-07:00"}

Document 2:
  ID: 558ee12c-2437-4e76-8dae-0a1cb63c0742
  User ID: amartays
  Memory: {"user_id": "amartays", "data": "Likes luxury hotels with spa services", "hash": "e4c2a6eb0311bbbed2ea174d6ccf00f5", "created_at": "2025-09-25T22:34:36.022239-07:00"}

Document 3:
  ID: 368f9cb4-502c-4ea0-8bbf-fb29e4a7c886
  User ID: amartays
  Memory: {"user_id": "amartays", "data": "Likes sea-facing hotels", "hash": "a9d18364bd807d4f9bcbe21156989f8e", "created_at": "2025-09-25T22:35:44.813170-07:00"}
